# Plot Extracted File in 2 columns

- adaptation : Sylvie Dagoret-Campagne
- creation date : 2025-07-03

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import getCalspec

%matplotlib widget 

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)

In [ ]:
REPOMAIN = False

In [ ]:
import lsst.daf.butler as dafButler

if REPOMAIN:
    repo = "/repo/main"
else:
    repo = "/repo/embargo"
    

In [ ]:
# Find collection in Butler /repo/embargo

if REPOMAIN:
    #my_collection = ['u/dagoret/auxtel_run_20250702b']
    #my_collection = ['u/dagoret/auxtel_run_20250702a']
    my_collection = ['u/dagoret/auxtel_run_20250703a']
else:
    my_collection = ['u/dagoret/auxtel_run_20250625a']


# save extraction
if REPOMAIN:
    #file_save = "auxtel_atmosphere_20250702b_repomain_v1.npy"
    #file_save = "auxtel_atmosphere_20250702a_repomain_v1.npy"
    file_save = "auxtel_atmosphere_20250703a_repomain_v1.npy"

else:    
    file_save = "auxtel_atmosphere_20250625a_v1.npy"

## Plots

In [ ]:
def append_field(recarray, name, data, dtype=None):
    """Ajoute une colonne à un numpy.recarray"""
    if dtype is None:
        dtype = data.dtype
    new_dtype = recarray.dtype.descr + [(name, dtype)]
    new_rec = np.empty(recarray.shape, dtype=new_dtype)
    for name_ in recarray.dtype.names:
        new_rec[name_] = recarray[name_]
    new_rec[name] = data
    return new_rec

In [ ]:
rec = np.load(file_save, allow_pickle=True)

# Conversion propre (suppression de timezone)
date_obs_clean = pd.to_datetime(rec["DATE-OBS"])

# Ajout au recarray
rec = append_field(rec, "DATE-OBS_CLEAN", date_obs_clean.to_numpy())

In [ ]:
df = pd.DataFrame(rec)
pd.set_option('display.max_columns', None)
print(rec.shape)

In [ ]:
df.head()

In [ ]:
ncols=2
nrows = 5

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["D2CCD", "PIXSHIFT", "PSF_REG", "CHI2_FIT", "OUTPRESS", "OUTTEMP", "alpha_0_2", "TARGETX", "TARGETY"]):

    ax = axs[idx]
 
    if col not in df.columns: 
        continue
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"

        
   

    if col_err in df.columns:
        ax.errorbar(rec["DATE-OBS"], rec[col], yerr=rec[col_err], linestyle="none", marker="+")
    else:
        ax.plot(rec["DATE-OBS"], rec[col], linestyle="none", marker="+")
        
    
    #plt.ylim((0.9*np.min(rec[col]), 1.1*np.max(rec[col])))
    if "PSF_REG" in col:
        plt.yscale("log")
        
    ax.grid()
    ax.set_title(col)
    #plt.legend()
    plt.gcf().autofmt_xdate()
    
plt.show()

## Filtered

In [ ]:
filtered = (rec["CHI2_FIT"] < 30) & (rec["PSF_REG"] > 1e-2) & (rec["D2CCD"] > 186.7)  & (rec["D2CCD"] < 187.4)  & (rec["PIXSHIFT"] > 0.5)  & (rec["PIXSHIFT"] < 1.5) 
print(len(filtered[filtered]))
filtered = filtered & (rec["PWV [mm]_err_x"] > 0) & (rec["PWV [mm]_err_x"] < 5) & (rec["PWV [mm]_err_y"] > 0) & (rec["PWV [mm]_err_y"] < 5)

In [ ]:
len(filtered[filtered])

In [ ]:
filtered = np.full(rec["CHI2_FIT"].shape,True)

In [ ]:
np.sum(filtered)

In [ ]:
ncols=2
nrows = 5

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["D2CCD", "PIXSHIFT", "PSF_REG", "CHI2_FIT", "OUTPRESS", "OUTTEMP", "OUTHUM", "alpha_0_2", "TARGETX", "TARGETY"]):

    ax = axs[idx]
    
    if col not in df.columns: 
        continue
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"

    
    if col_err in df.columns:
        ax.errorbar(rec["DATE-OBS"][filtered], rec[col][filtered], yerr=rec[col_err][filtered], linestyle="none", marker="+")
    else:
        ax.plot(rec["DATE-OBS"][filtered], rec[col][filtered], linestyle="none", marker="+")
        
    ax.set_ylim((0.9*np.nanmin(rec[col][filtered]), 1.1*np.nanmax(rec[col][filtered])))
    if "PSF_REG" in col:
        plt.yscale("log")
    ax.grid()
    ax.set_title(col)
    ax.legend()
    plt.gcf().autofmt_xdate()
plt.show()

### Spectrogram left , Spectrum right fits

In [ ]:
ncols=2
nrows = 10

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["A1_x","A1_y", "chi2_x", "chi2_y", "ozone [db]_x","ozone [db]_y", "PWV [mm]_x","PWV [mm]_y", "VAOD_x","VAOD_y", "A2_x","A2_y", "D_CCD [mm]_x","D_CCD [mm]_y", "alpha_pix [pix]", "reso [nm]","B_x" ,"B_y"]):

    ax = axs[idx]
    
    if col not in df.columns: 
        continue
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"
    
    if col_err in df.columns:
        ax.errorbar(rec["DATE-OBS"][filtered], rec[col][filtered], yerr=rec[col_err][filtered], linestyle="none", marker="+")
    else:
        ax.plot(rec["DATE-OBS"][filtered], rec[col][filtered], linestyle="none", marker="+")
    ax.set_ylim((0.9*np.min(rec[col][filtered]), 1.1*np.max(rec[col][filtered])))
    ax.grid()
    ax.set_title(col)
    ax.legend()
    plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
filters = np.unique(rec["FILTER"])
filters

In [ ]:
ncols=2
nrows = 10

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["A1_x","A1_y", "chi2_x", "chi2_y", "ozone [db]_x","ozone [db]_y", "PWV [mm]_x","PWV [mm]_y", "VAOD_x","VAOD_y", "A2_x","A2_y", "D_CCD [mm]_x","D_CCD [mm]_y", "alpha_pix [pix]", "reso [nm]","B_x" ,"B_y"]):

    ax = axs[idx]


    if col not in df.columns: 
        continue
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"

    
    for filt in filters:
        if filt in ["HD60753", "HD37962"]:
            continue
        index = filtered & (rec["FILTER"] == filt)
        if col_err in df.columns:
            ax.errorbar(rec["DATE-OBS"][index], rec[col][index], yerr=rec[col_err][index], linestyle="none", marker="+", label=filt)
        else:
            ax.plot(rec["DATE-OBS"][index], rec[col][index], linestyle="none", marker="+",label=filt)
    ax.set_ylim((0.9*np.min(rec[col][filtered]), 1.1*np.max(rec[col][filtered])))
    ax.grid()
    ax.set_title(col)
    ax.legend()
    plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
stars = np.unique(rec["TARGET"])
stars

In [ ]:
ncols=2
nrows = 12

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["A1_x","A1_y", "ozone [db]_x", "ozone [db]_y", "PWV [mm]_x","PWV [mm]_y","VAOD_x" ,"VAOD_y", "angstrom_exp_x","angstrom_exp_y","A2_x" ,"A2_y", "D_CCD [mm]_x","D_CCD [mm]_y", "alpha_pix [pix]", "reso [nm]","B_x" ,"B_y", "alpha_0_2", "alpha_0_1", "gamma_0_2", "gamma_0_1", "y_c_0_2", "y_c_0_1"]):

    ax = axs[idx]
    
    if col not in df.columns: 
        continue
        
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"

    
    for star in stars:
        #if star not in ["HD185975"]:
        #    continue
        index = filtered & (rec["TARGET"] == star)
        if col_err in df.columns and False:
            ax.errorbar(rec["DATE-OBS"][index], rec[col][index], yerr=rec[col_err][index], linestyle="none", marker="+", label=star)
        else:
            ax.plot(rec["DATE-OBS"][index], rec[col][index], linestyle="none", marker="+",label=star)
    ax.set_ylim((0.9*np.nanmin(rec[col][filtered]), 1.1*np.nanmax(rec[col][filtered])))
    ax.grid()
    ax.set_title(col)
    ax.legend()
    #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #fig.autofmt_xdate()
    plt.gcf().autofmt_xdate()
  
plt.show()

### Spectrogram forward model fits

In [ ]:
filters = np.unique(rec["FILTER"])
filters

In [ ]:
ncols=2
nrows = 5

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()


for idx,col in enumerate(["A1_x", "A1_y","ozone [db]_x", "ozone [db]_y","PWV [mm]_x", "PWV [mm]_y","VAOD_x", "VAOD_y","D_CCD [mm]_x","D_CCD [mm]_y"]): #, "gamma_0", "alpha_0"]:

    ax = axs[idx]
    
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"

    
    for filt in filters:
        index = filtered & (rec["FILTER"] == filt)
        if col_err in df.columns:
            ax.errorbar(rec["DATE-OBS"][index], rec[col][index], yerr=rec[col_err][index], linestyle="none", marker="+",label=filt)
        else:
            ax.plot(rec["DATE-OBS"][index], rec[col][index], linestyle="none", marker="+",label=filt)
            
    ax.set_ylim((0.9*np.min(rec[col][filtered]), 1.1*np.max(rec[col][filtered])))
    ax.grid()
    ax.set_title(col)
    ax.legend()
    plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
stars = np.unique(rec["TARGET"])
#stars = ['HD2811', 'HD38666']  # , 'HD185975'
stars

In [ ]:
ncols=2
nrows = 7

fig,axes  = plt.subplots(nrows,ncols,figsize=(16,nrows*4),layout="constrained")

axs = axes.flatten()

for idx,col in enumerate(["A1_x", "A1_y","ozone [db]_x", "ozone [db]_y","PWV [mm]_x", "PWV [mm]_y", "VAOD_x", "VAOD_y","angstrom_exp_x", "angstrom_exp_y", "D_CCD [mm]_x", "D_CCD [mm]_y","gamma_0_2", "alpha_0_2"]):

    ax = axs[idx]
    
    if len(col.split('_')) > 1:
        col_err = '_'.join(col.split('_')[:-1])+"_err_"+col.split('_')[-1]
    else:
        col_err = col+"_err"
        
    
    for star in stars : #['HD185975']: #stars:
        index = filtered & (rec["TARGET"] == star)
        if not getCalspec.is_calspec(star):
            marker = "o"
        else:
            marker = "+"
            
        if col_err in df.columns:
            ax.errorbar(rec["DATE-OBS"][index], rec[col][index], yerr=rec[col_err][index], linestyle="none", marker=marker, label=star)
        else:
            ax.plot(rec["DATE-OBS"][index], rec[col][index], linestyle="none", marker=marker,label=star)
    ax.set_ylim((0.9*np.min(rec[col][filtered]), 1.1*np.max(rec[col][filtered])))
    ax.grid()
    ax.set_title(col)
    ax.legend()
    plt.gcf().autofmt_xdate()
    #plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #fig.autofmt_xdate()
    #fig.tight_layout()
plt.show()